In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from moviepy.editor import VideoFileClip
import os
import csv
from openpyxl import Workbook

class VideoPointExtractor:
    def __init__(self, root):
        self.root = root
        self.root.title("Video Point Extractor")

        # Se inicializa el canvas sin tamaño específico, se ajustará al tamaño del video
        self.canvas = tk.Canvas(root)
        self.canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.video_files = []
        self.current_video_index = -1
        self.coordinates = []

        # Configura los comandos de teclado
        self.root.bind("<Key>", self.on_key_press)
    
    def on_key_press(self, event):
        # Verifica el carácter en minúscula para manejar las teclas
        if event.char.lower() == 'l':
            self.load_videos()
        elif event.char.lower() == 's':
            self.save_coordinates()
        elif event.char.lower() == 'n':
            self.next_video()
        elif event.char.lower() == 'q':
            if self.current_video_index == len(self.video_files) - 1:
                self.root.quit()
        elif event.keysym == 'Escape':
            self.root.quit()

    def load_videos(self):
        folderpath = filedialog.askdirectory()
        if folderpath:
            self.video_files = [os.path.join(folderpath, f) for f in os.listdir(folderpath) if f.endswith(('.mp4', '.avi', '.mov'))]
            if self.video_files:
                self.current_video_index = 0
                self.show_video(self.current_video_index)
            else:
                messagebox.showinfo("No Videos Found", "No video files found in the selected directory.")

    def show_video(self, index):
        self.current_video_index = index
        self.canvas.delete("all")  # Limpiar el lienzo
        video_clip = VideoFileClip(self.video_files[index])
        frame = video_clip.get_frame(0)
        self.image = Image.fromarray(frame)

        self.canvas.config(width=video_clip.size[0], height=video_clip.size[1])
        self.photo = ImageTk.PhotoImage(self.image)
        self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)

        self.canvas.bind("<Button-1>", self.on_canvas_click)

    def next_video(self):
        if self.current_video_index + 1 < len(self.video_files):
            self.save_coordinates()  # Guardar coordenadas antes de pasar al siguiente video
            self.show_video(self.current_video_index + 1)
        else:
            messagebox.showinfo("End of Videos", "No more videos available.")

    def on_canvas_click(self, event):
        # Marcar el punto en el lienzo
        x, y = event.x, event.y
        self.canvas.create_oval(x-3, y-3, x+3, y+3, fill="red")
        self.coordinates.append((x, y))

    def save_coordinates(self):
        if self.coordinates:
            video_file = self.video_files[self.current_video_index]
            video_base_name = os.path.splitext(os.path.basename(video_file))[0]
            output_filename = os.path.join(os.path.dirname(video_file), f"{video_base_name}.xlsx")

            workbook = Workbook()
            sheet = workbook.active
            sheet.append(['Coordenada X', 'Coordenada Y'])
            for coord in self.coordinates:
                sheet.append([coord[0], coord[1]])

            workbook.save(output_filename)

            messagebox.showinfo("Saved", f"Coordinates saved to {output_filename}")
            self.coordinates = []  # Limpiar las coordenadas después de guardar

if __name__ == "__main__":
    root = tk.Tk()
    app = VideoPointExtractor(root)
    root.mainloop()